In [2]:
import xml.etree.ElementTree as ET
import requests
import psycopg2
import os
from dotenv import load_dotenv

# Load environment variables (should be identical to other scripts)
load_dotenv()

# Fetch the XML data from the URL
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/lfsq_esgan2/1.0?compress=false"
response = requests.get(url)
xml_data = response.text if response.status_code == 200 else None

# Define namespaces 
namespaces = {
    'm': 'http://www.sdmx.org/resources/sdmxml/schemas/v3_0/message',
}

# Parse the XML data
root = ET.fromstring(xml_data)

# Find the <m:DataSet> element
data_set = root.find('.//m:DataSet', namespaces)

# Extracting data from XML file
structured_data = []
for series in data_set.findall('.//Series', namespaces):
    series_data = series.attrib
    for obs in series.findall('.//Obs', namespaces):
        if 'OBS_VALUE' in obs.attrib and obs.attrib['OBS_VALUE']:
            obs_data = obs.attrib
            record = {**series_data, **obs_data}
            structured_data.append(record)

# Raw data
data = structured_data

# Define the database configurations for each environment
db_configs = [
    {
        'dbname': 'Antigone',
        'user': os.getenv('DB_USER', 'DB_USER'),
        'password': os.getenv('DB_PASSWORD', 'DB_PASSWORD'),
        'host': os.getenv('DB_HOST', 'DB_HOST'),
        'schema': 'Source'
    },
    {
        'dbname': 'Ismene',
        'user': os.getenv('DB_USER', 'DB_USER'),
        'password': os.getenv('DB_PASSWORD', 'DB_PASSWORD'),
        'host': os.getenv('DB_HOST', 'DB_HOST'),
        'schema': 'Source'
    },
    {
        'dbname': 'Eteocles',
        'user': os.getenv('DB_USER', 'DB_USER'),
        'password': os.getenv('DB_PASSWORD', 'DB_PASSWORD'),
        'host': os.getenv('DB_HOST', 'DB_HOST'),
        'schema': 'Source'
    },
    {
        'dbname': 'Polyneices',
        'user': os.getenv('DB_USER', 'DB_USER'),
        'password': os.getenv('DB_PASSWORD', 'DB_PASSWORD'),
        'host': os.getenv('DB_HOST', 'DB_HOST'),
        'schema': 'Source'
    }
]

# Iterate over each database configuration
for config in db_configs:
    # Connect to the database
    conn = psycopg2.connect(
        dbname=config['dbname'],
        user=config['user'],
        password=config['password'],
        host=config['host'],
        options=f"-c search_path={config['schema']}"
    )
    cur = conn.cursor()

    # Table name, could be modified based on file name or other logic if needed
    table_name = f'"{config["schema"]}".self_employment_by_industry_raw'

    # SQL to create a table if it doesn't exist
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        age TEXT,
        freq TEXT,
        geo TEXT,
        nace_r2 TEXT,
        sex TEXT,
        unit TEXT,
        wstatus TEXT,
        time_period TEXT,
        obs_value NUMERIC,
        obs_flag TEXT
    );
    """
    cur.execute(create_table_query)

    # SQL insert statement
    insert_stmt = f"""
    INSERT INTO {table_name} (age, freq, geo, nace_r2, sex, unit, wstatus, time_period, obs_value, obs_flag)
    VALUES (%(age)s, %(freq)s, %(geo)s, %(nace_r2)s, %(sex)s, %(unit)s, %(wstatus)s, %(TIME_PERIOD)s, %(OBS_VALUE)s, %(OBS_FLAG)s);
    """

    # Insert the data into the database
    for record in data:
        record_with_defaults = {
            'age': record.get('age'),
            'freq': record.get('freq'),
            'geo': record.get('geo'),
            'nace_r2': record.get('nace_r2'),
            'sex': record.get('sex'),
            'unit': record.get('unit'),
            'wstatus': record.get('wstatus'),
            'TIME_PERIOD': record.get('TIME_PERIOD'),
            'OBS_VALUE': record.get('OBS_VALUE'),
            'OBS_FLAG': record.get('OBS_FLAG', 'na')  # Provide a default value
        }
        cur.execute(insert_stmt, record_with_defaults)

    # Commit the transaction and close the connection
    conn.commit()
    cur.close()
    conn.close()
    print(f"Data inserted into {config['dbname']} successfully.")


Data inserted into Antigone successfully.
Data inserted into Ismene successfully.
Data inserted into Eteocles successfully.
Data inserted into Polyneices successfully.
